# DDPG

Ene shit ni bol Q-value technique-iig gradient policy-toi hamtatgaj heregleed , iluu huchtei bolson shit.

## Importing the libraries

In [14]:
!pip install gymnasium
!pip install swig
!pip install "gymnasium[box2d]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376104 sha256=612494890e3418a2667b0ea8cd5d2f47372b2157e177c48641ebaf0bd76d22fe
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py


In [15]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from collections import deque
import torch.optim as optim

## Setting the hyperparameters

In [16]:
state_dim = 96*96*3
action_dim = 3
hidden_dim=256
batch_size = 128
learning_rate = 1e-4
gamma = 0.99
tau=0.005
buffer_size = int(1e5)
min_buffer_size = 1000
num_episodes=500
max_timesteps=1000

## Building Actor Network

In [17]:
class Actor(nn.Module):
    def __init__(self):
        super(Actor,self).__init__()
        self.fc1=nn.Linear(state_dim,hidden_dim)
        self.fc2=nn.Linear(hidden_dim,hidden_dim)
        self.fc3=nn.Linear(hidden_dim,action_dim)
    def forward(self,state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        action = torch.tanh(self.fc3(x)) # Tanh for bounded action
          # bounded action ni todorhoi range-d l baih action gesen ug.
        return action


## Building the Critic network

In [18]:
class Critic(nn.Module):
    def __init__(self):
        super(Critic,self).__init__()
        self.fc1 = nn.Linear(state_dim+action_dim,hidden_dim) # state and action gesen "pair" yavdag tul , end nemj baina.
        self.fc2 = nn.Linear(hidden_dim,hidden_dim)
        self.fc3 = nn.Linear(hidden_dim,1)
    def forward(self,state,action):
        x = F.relu(self.fc1(torch.cat([state,action],dim=1))) # concatenate hiichij baigaa.
        x = F.relu(self.fc2(x))
        value = self.fc3(x)
        return value

## Implementing the Replay Buffer

In [19]:
class ReplayBuffer():
    def __init__(self,capacity):
        self.buffer = deque(maxlen=capacity)
    def add(self,state,action,reward,next_state,done):
        self.buffer.append((state,action,reward,next_state,done))
    def sample(self,batch_size):
        return random.sample(self.buffer,batch_size)
    def __len__(self):
        return len(self.buffer)


## Implementing the DDPG Algorithm

In [33]:
class DDPG :
    def __init__(self):
        self.actor = Actor()
        self.actor_target = Actor()
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(),lr=learning_rate)
        self.critic = Critic()
        self.critic_target = Critic()
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(),lr=learning_rate)
        self.replay_buffer = ReplayBuffer(buffer_size)

    def update(self,batch_size):
        if len(self.replay_buffer) < min_buffer_size:
            return  # Exit if not enough samples
        samples = self.replay_buffer.sample(batch_size)
        states,actions,rewards,next_states,dones = map(torch.tensor,zip(*samples))
        # Update the Critic network:
        with torch.no_grad():
            next_actions = self.actor_target(next_states)
            Q_target_next = self.critic_target(next_states,next_actions) # Get Q-value for next state-action pair from target critic network
            Q_targets = rewards + (gamma*Q_target_next*(1-dones))   # Q_target is the sum of immediate reward and discounted future reward
        Q_expected = self.critic(states,actions)
        critic_loss = F.mse_loss(Q_expected,Q_targets)
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        # Update of the Actor network
        actor_loss = -self.critic(states,self.actor(states)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        # Soft update of the Critic Target and the Actor target networks
        self.soft_update(self.critic,self.critic_target)
        self.soft_update(self.actor,self.actor_target)
    @staticmethod
    def soft_update(local_model,target_model):
        for target_param,local_param in zip(target_model.parameters(),local_model.parameters()):
            target_param.data.copy_(tau*local_param.data+(1.0-tau)*target_param.data)


## Implementing the Training Loop

In [34]:
def train(env,agent,num_episodes,max_timesteps):
    for episode in range(num_episodes):
        state,_ = env.reset()
        state = torch.tensor(state.flatten(),dtype=torch.float32) # Flatten state
        episode_reward = 0
        for t in range(max_timesteps):
            action = agent.actor(state).detach().numpy()
            next_state,reward,done,_ = env.step(action)
            next_state = torch.tensor(next_state.flatten(),dtype=torch.float32)
            agent.replay_buffer.add(state,action,reward,next_state,done)
            state = next_state
            episode_reward += reward
            agent.update(batch_size)
            if done:
                break
        print(f"Episode {episode}: Reward = {episode_reward}")


## Doing other shit

In [35]:
# Setting up the environment
env = gym.make("CarRacing-v3")
# Creating the agent
ddpg_agent = DDPG()
# Starting the training
train(env,ddpg_agent,num_episodes,max_timesteps)

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray